In [62]:
import numpy as np #Biblioteka służąca do obliczeń
import math #funkcje matematyczne sqrt/ potęgi itd.
import random #liczby pseudolosowe

In [445]:
def function(arr):
    max = 0
    index = 0
    for i in range(0,len(arr)):
        if(arr[i]>max):
            max = arr[i]
            index = i
        
    arr[index] = 1

    for i in range(0,len(arr)):
        if(arr[i]<1):
            arr[i]=0

    return arr
    
class NeuralNetwork:
    def __init__(self,sizes): 
        self.num_layers=len(sizes)
        self.sizes=sizes
        self.weights=[np.random.randn(x,y) for x,y in zip(sizes[:-1],sizes[1:])]
        
    def feedforward(self,sample):
        #Wagi
        for w in self.weights:
            sample=activationFunction(np.dot(sample,w))
        return sample    
       
    def resetWeights(self):
        self.weights = [np.random.randn(x,y) for x,y in zip(sizes[:-1],sizes[1:])]
    
    def setWeights(self, weightsFromHeuristic): #pobierac wektor wag poprze heurystyke i wpisywac go do self.weights 
        for i in range(0, len(self.weights)):
            for j in range(0, len(self.weights[i])):
                for k in range(0, len(self.weights[i][j])):
                    self.weights[i][j][k] = self.weights[i][j][k] * weightsFromHeuristic
    
    def error(self,sample):
        processedSample = self.feedforward(sample[:4])
       
        if(sample[4:7] == [1,0,0]): 
            flowerList = [1,0,0]
        elif(sample[4:7] == [0,1,0]):
            flowerList = [0,1,0]
        elif(sample[4:7] == [0,0,1]):
            flowerList = [0,0,1]
        
        sum = 0
        #Błąd średniokwadratowy
        for i in range(0, len(processedSample)):
            sum += (processedSample[i] - flowerList[i])**2
        
        #Zwrócenie errora dla pojedynczej probki
        return sum
        

def activationFunction(x):
    value = 1.0/(1.0+np.exp(-x))#sigmoid function
    return value

def globalErrorOfNN(weights,trainSet):
    #stworzenie sieci neuronowej - obiekt sieci
    errorWeight = []
    netNetwork = NeuralNetwork([4,3,3])
    
    errors=[None] * len(trainSet)
    
    rand = random.randint(0, len(weights)-1)
    weight = weights[rand]
    
    #ustawienie wag z weights do sieci dzieki setWeights()
    netNetwork.setWeights(weight)
    
    #Obliczenie błędu dla pojedynczej próbki
    i=0
    for rekord in trainSet:
        errors[i] = netNetwork.error(rekord)
        i = i+1
    #Błąd sieci
    netError = sum(errors)/len(errors)
    
    #Dodanie wagi i  errora
    errorWeight.append(netError)
    errorWeight.append(weight)
    #Średni error
    return errorWeight

def makeDecision(validationSet, best_Weight):
    corrected = 0
    net = NeuralNetwork([4,3,3])
    net.setWeights(best_Weight)
    answerSet = []
    resaultSet = []
    compareSet = []
    
    k=0
    for k in range(0, len(validationSet)):
        rekord = validationSet[k]
        compareSet.append(rekord[4:7])
        k = k +1
        
    
    i = 0
    for i in range(0, len(validationSet)):
        rekord = validationSet[i]
        answerSet.append(net.feedforward(rekord[:4]))
        i = i + 1
    
    for rekord in answerSet:
        resaultSet.append(function(rekord))
        
    corrected = 0
    for z in range(0, len(compareSet)):
        if((compareSet[z] == resaultSet[z]).all()):
            corrected = corrected +1

    accuracy = corrected/len(compareSet)*100
    print(f'Skuteczność na poziomie: {accuracy} %'.format(accuracy))
    



In [241]:
#Klasa cząstki
class Particle:
    def __init__(self, x0):
        self.position = [] #pozycja cząstki
        self.velocity = [] #prędkość cząstki
        self.post_best = [] #najlepsza pozycja cząstki
        self.err_best = -1 #najlepszy błąd cząstki
        self.err = -1 #najlepszy błąd
        
        
        for i in range(0, num_dimensions): #od 0 do ilości wymiarów
            self.velocity.append(random.uniform(-1,1)) #losujemy wektor prędkości dla cząstki
            self.position.append(x0[i]) #przypisujemy określoną pozycję dla cząstk
    
    #obliczamy obecny fitness dla konkretnej pozycji cząstki
    def evaluate(self, costFunc):
        self.err = costFunc(self.position)
        
        #sprawdzamy czy obecna pozycja jest najlepszą indywidualną pozycją
        if self.err < self.err_best or self.err_best == -1: #jezeli jest rowny -1 to jeszcze nie ulegl zmianie
            self.post_best = self.position #przypisanie nowej najlepszej pozycji
            self.err_best = self.err #przypisanie nowego błędu
    
    #aktualizujemy prędkość cząstki
    def update_velocity(self, post_best_g):
        w = 0.5 #stala do mnożnika
        c1 = 1 #1 składowa wektora prędkości
        c2 = 2 #2 składowa wektora prędkości
        
        for i in range(0, num_dimensions):
            r1 = random.random() #losowa wielkość dla pierwszej składowej wektora
            r2 = random.random() #losowa wielkość dla 2 składowej wektora
            
            vel_cognitive=c1*r1*(self.post_best[i]-self.position[i]) #1 składowa
            vel_social=c2*r2*(post_best_g[i]-self.position[i])       #2 składowa
           
            self.velocity[i]=w*self.velocity[i]+vel_cognitive+vel_social #zapamiętanie prędkości
    
    #aktualizujemy pozycje cząstki
    def update_position(self, bounds):
        for i in range(0, num_dimensions):
            self.position[i] = self.position[i] + self.velocity[i] #przemieszczenie cząstki zgodne z wektorem jej aktualnej prędkości
            
            #jeżeli potrzebne to dodajemy maksymalną granicę
            if self.position[i] > bounds[i][1]:
                self.position[i] = bounds[i][1] #jeżeli przekroczyliśmy to ustawiamy na maksymalny zakres granic
 
            #jeżeli potrzebnme to dodajemy minimalną granicę
            if self.position[i] < bounds[i][0]:
                self.position[i] = bounds[i][0]

In [242]:
def cost_function(x):
    #Błąd średniokwadratowy
    total = 0 #koszt
    for i in range(len(x)):
        total+=x[i]**2
    return total #zwracamy koszt

#Klasa Algorytmu PSO (Particle swarm optimization - Algorytm roju cząstek)
class PSO:
    def __init__(self, costFunc, x0, bounds, num_particles, maxiter):
        global num_dimensions
        
        num_dimensions = len(x0) #liczba rozmiarów jest równa długości ilości wymiarów przestrzeni
        err_best_g = -1 #najlepszy error dla grupy
        pos_best_g = [] #najlepsza pozycja dla grupy
        self.weights_From_Heuristic = [] #wagi od heursytyki
        
        #zakładamy rój
        swarm = []
        for i in range(0, num_particles):
            swarm.append(Particle(x0)) #tworzymy obiekt cząstki z klasy wyżej i dodajemy go do roju
            
        #Pętla optymalizacyjna
        i = 0
        while i < maxiter:
            #Iterujemy przez cząstki i obliczamy dla nich funkcję fitness
            for j in range(0,num_particles):
                swarm[j].evaluate(costFunc) #obliczamy wartość funkcji dla j-tej cząstki w roju
                
                
                #Sprawdzamy czy obecna cząstka jest najlepszą cząstka globalną
                if swarm[j].err < err_best_g or err_best_g == -1: 
                    pos_best_g = list(swarm[j].position) #zwraca listę w tym wypadku zawierająca wsp pozycji j-tej cząstki w roju
                    err_best_g = float(swarm[j].err)
                    self.weights_From_Heuristic.append(err_best_g)
                
            
            #Iterujemy przez rój i aktualizyjemy pozycje oraz ich wektory prędkości
            for j in range(0, num_particles):
                swarm[j].update_velocity(pos_best_g) #aktualizujemy prędkość
                swarm[j].update_position(bounds) #aktualizyjemy pozycję
            
            i+=1
            
            
            
                
        
        

In [243]:
def readDataBaseCsv():
    dane = []
    f = open("iris.csv", "r")
    for line in f:
        dane.append([float(s) for s in line.split(",")])
    random.shuffle(dane)
    return dane

def split(base):
    brd = math.ceil(.7*len(base))
    train = base[:brd]
    val = base[brd:]
    return train,val

In [244]:
#Obsluga heurystyki, w celu pozyskania wag
initial = [10,10]
weights = []
bounds=[(-10,10), (-10,10)]
#Podajemy funkcję do optymalizacji, początkowe współrzędne, granice przestrzeni, liczbę cząstek oraz ilość iteracji
weights = PSO(cost_function, initial, bounds, num_particles=15, maxiter=30) 

In [245]:
#Obsługa bazy danych
base = readDataBaseCsv()
sums = [None] * len(baza)
sizes = [4,3,3]
net = NeuralNetwork(sizes)
trainBase, validationBase = split(base)

In [429]:
#Szukanie najmniejszego błedu w bazie treningowej
i = 0
res = []
for elem in trainBase:
    net.resetWeights()
    res.append(globalErrorOfNN(weights.weights_From_Heuristic, trainBase))
    i = i +1

#Znalezienie indexu i minimalnego błędu
min_error = 200
weight = 0
for i in range(0, len(res)):
    if min_error > res[i][0]:
        min_error = res[i][0]
        weight = res[i][1]

print(min_error)
print(weight)

0.6291933248991084
4.0749066840463675


<ipython-input-424-8a276610f5f3>:58: RuntimeWarning: overflow encountered in exp
  value = 1.0/(1.0+np.exp(-x))#sigmoid function


In [444]:
makeDecision(validationBase, weight) 

Skuteczność na poziomie: 53.333333333333336 %
